# Experiment 4 – Simulate Data Acces Layer (DAL) for write

## Setup

In [1]:
from dotenv import load_dotenv
from constants import *
from utility_functions import *
from constants import *
from DataAccessLayerClient import DataAccessLayerClient
import pyarrow.parquet as pq

# Load environment variables
load_dotenv()

True

In [2]:
# Setup for connection towards AWS KMS
encryption_config = pe.EncryptionConfiguration(
    footer_key="file-access-key",             
    column_keys={
        "salary-key": ["Salary"],     
        "password-key": ["Password"] 
    },
    plaintext_footer=False         
)
kms_conn_config = pe.KmsConnectionConfig(
    kms_instance_id=f"aws-kms-{REGION}",
    kms_instance_url=f"https://kms.{REGION}.amazonaws.com"
)

In [3]:
dal = DataAccessLayerClient(base_url="dal://rbac-sensitive/")

In [4]:
# Get credentials for roles
engineer_credentials = get_aws_credentials(ROLE_ENGINEER)
hr_credentials = get_aws_credentials(ROLE_HR)
admin_credentials = get_aws_credentials(ROLE_ADMIN)

# Get aws client for S3 and KMS
engineer_kms_client = create_aws_client_for(engineer_credentials, "kms")
hr_kms_client = create_aws_client_for(hr_credentials, "kms")
admin_kms_client = create_aws_client_for(admin_credentials, "kms")

admin_s3_client = create_aws_client_for(admin_credentials, "s3")

Assumed ENGINEER role successfully.
Assumed HR role successfully.
Assumed ADMIN role successfully.


In [11]:
# Make sure the encrypted employee data is fresh

admin_s3_client.upload_file(EMPLOYEE_DATA_ENCRYPTED_PATH, BUCKET_NAME, EMPLOYEE_DATA_ENCRYPTED_KEY)

In [6]:
file = get_data(admin_s3_client, EMPLOYEE_DATA_ENCRYPTED_KEY)
df = decrypt_parquet(file, admin_kms_client)
print(f"Decrypted first row before change:\n{df.head(1)}")


 successfully loaded 'employee_data_encrypted.parquet' from S3 bucket 's3-rbac-in-data-lakes-experiments'
File columns: ['ID', 'Name', 'Email', 'Department', 'Salary', 'Password']
Decrypted first row before change:
   ID   Name              Email Department  Salary        Password
0   1  Alice  alice@example.com         HR   55000  DummyPassword1


In [7]:
df.loc[0, "Salary"] = 100000

In [8]:
print(f"Decrypted first row before change:\n{df.head(1)}")

Decrypted first row before change:
   ID   Name              Email Department  Salary        Password
0   1  Alice  alice@example.com         HR  100000  DummyPassword1


## Experiment: Try to upload modified file to s3 for different roles 

In [9]:
dal.put_data(token=TOKEN_ADMIN, data=df, file_name=EMPLOYEE_DATA_ENCRYPTED_KEY)


Authenticating user with token: ADMIN_TOKEN


Assumed ADMIN role successfully.
Successfully uploaded 'employee_data_encrypted.parquet' to S3 bucket 's3-rbac-in-data-lakes-experiments'


In [10]:
# Get data again to verify change
df = dal.get_data(token=TOKEN_ADMIN, file_name = EMPLOYEE_DATA_ENCRYPTED_KEY)
print(f"Decrypted column new fetch:\n{df.head(1)}")

Authenticating user with token: ADMIN_TOKEN
Assumed ADMIN role successfully.

 successfully loaded 'employee_data_encrypted.parquet' from S3 bucket 's3-rbac-in-data-lakes-experiments'
File columns: ['ID', 'Name', 'Email', 'Department', 'Salary', 'Password']
Decrypted column new fetch:
   ID   Name              Email Department  Salary        Password
0   1  Alice  alice@example.com         HR  100000  DummyPassword1
